## Joining on Non-Equi Operators

In [1]:
import pandas as pd
import janitor
import numpy as np 

/opt/homebrew/Caskroom/miniforge/base/envs/pyjanitor-dev/lib/python3.9/site-packages/pandas_flavor/register.py:31: UserWarning: registration of accessor <class 'pandas_flavor.register.register_dataframe_method.<locals>.inner.<locals>.AccessorMethod'> under name 'then' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  register_dataframe_accessor(method.__name__)(AccessorMethod)


In [2]:
url = "https://raw.githubusercontent.com/samukweku/data-wrangling-blog/master/notebooks/Data_files/flights.csv"

flights = pd.read_csv(url, sep = '\t', names=['orig','dest','orig_time', 'dest_time'], parse_dates  = ['orig_time', 'dest_time'])
flights = flights.factorize_columns(['orig','dest']).iloc[:, 2:]
flights.columns = [ent.split("_")[0] if ent.endswith('enc') else ent for ent in flights]
flights.columns = ['takeoff','landing','orig','dest']
flights = flights.assign(start=flights.landing+pd.Timedelta(minutes=45), end=flights.landing+pd.Timedelta(hours=3))
flights


,takeoff,landing,orig,dest,start,end
0,2021-11-27 07:15:00,2021-11-27 08:55:00,0,0,2021-11-27 09:40:00,2021-11-27 11:55:00
1,2021-11-27 20:05:00,2021-11-28 00:50:00,1,1,2021-11-28 01:35:00,2021-11-28 03:50:00
2,2021-11-27 21:00:00,2021-11-27 21:35:00,2,2,2021-11-27 22:20:00,2021-11-28 00:35:00
3,2021-11-27 21:15:00,2021-11-27 22:25:00,0,2,2021-11-27 23:10:00,2021-11-28 01:25:00
4,2021-11-26 11:40:00,2021-11-26 14:45:00,3,3,2021-11-26 15:30:00,2021-11-26 17:45:00
...,...,...,...,...,...,...
127421,2022-01-29 13:45:00,2022-01-29 15:35:00,18,27,2022-01-29 16:20:00,2022-01-29 18:35:00
127422,2022-01-29 11:20:00,2022-01-29 12:20:00,27,2,2022-01-29 13:05:00,2022-01-29 15:20:00
127423,2022-01-29 16:00:00,2022-01-29 20:15:00,94,143,2022-01-29 21:00:00,2022-01-29 23:15:00
127424,2022-01-29 09:20:00,2022-01-29 10:55:00,27,27,2022-01-29 11:40:00,2022-01-29 13:55:00


In [3]:
%%timeit 
outt = (flights
        .merge(flights, left_on='dest', right_on='orig')
        .loc[lambda f: f.takeoff_y.between(f.start_x, f.end_x) & (f.orig_x != f.orig_y), 
             ['start_x', 'takeoff_y', 'start_y', 'dest_x', 'orig_y']
             ]
        )

outt

,start_x,takeoff_y,start_y,dest_x,orig_y
5465,2021-11-27 11:40:00,2021-11-27 11:55:00,2021-11-27 15:20:00,0,0
11139,2021-11-28 16:15:00,2021-11-28 18:30:00,2021-11-28 22:30:00,0,0
21919,2021-12-13 13:25:00,2021-12-13 13:35:00,2021-12-13 18:00:00,0,0
21931,2021-12-13 13:25:00,2021-12-13 14:15:00,2021-12-13 17:20:00,0,0
22070,2021-12-13 13:25:00,2021-12-13 13:50:00,2021-12-13 16:15:00,0,0
...,...,...,...,...,...
85248180,2022-03-21 09:25:00,2022-03-21 11:30:00,2022-03-21 13:35:00,193,193
85249542,2022-01-01 11:15:00,2022-01-01 11:20:00,2022-01-01 13:00:00,197,197
85249565,2022-01-01 10:50:00,2022-01-01 10:55:00,2022-01-01 12:25:00,197,197
85249566,2022-01-01 10:50:00,2022-01-01 11:20:00,2022-01-01 13:00:00,197,197


In [4]:
%%timeit
out = (flights
 .conditional_join(
     flights, 
     ('end', 'takeoff' ,'>='), 
    ('start', 'takeoff', '<='), 
     ('orig','orig','!='), 
     ('dest', 'orig', '=='), 
     df_columns = ['start', 'end', 'dest'],
     right_columns = ['takeoff', 'orig'],
     use_numba=True)
)
out


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


,start,end,dest,takeoff,orig
0,2021-11-27 17:10:00,2021-11-27 19:25:00,4,2021-11-27 19:15:00,4
1,2021-11-27 13:05:00,2021-11-27 15:20:00,6,2021-11-27 13:45:00,6
2,2021-11-27 11:45:00,2021-11-27 14:00:00,7,2021-11-27 12:45:00,7
3,2021-11-27 11:45:00,2021-11-27 14:00:00,7,2021-11-27 13:00:00,7
4,2021-11-27 11:45:00,2021-11-27 14:00:00,7,2021-11-27 13:40:00,7
...,...,...,...,...,...
83780,2022-01-29 09:50:00,2022-01-29 12:05:00,44,2022-01-29 12:05:00,44
83781,2022-01-29 14:30:00,2022-01-29 16:45:00,26,2022-01-29 14:45:00,26
83782,2022-01-29 10:10:00,2022-01-29 12:25:00,85,2022-01-29 11:25:00,85
83783,2022-01-29 21:00:00,2022-01-29 23:15:00,143,2022-01-29 22:00:00,143


In [5]:

%%timeit
outer = (flights
 .conditional_join(
     flights, 
     ('end', 'takeoff' ,'>='), 
     ('start', 'takeoff', '<='), 
     ('orig','orig','!='), 
    ('dest', 'orig', '=='), 
    df_columns = ['start', 'end', 'dest'],
     right_columns = ['takeoff', 'orig'],
     use_numba=False)
)
outer

,start,end,dest,takeoff,orig
0,2021-11-27 11:40:00,2021-11-27 13:55:00,0,2021-11-27 11:55:00,0
1,2021-11-28 16:15:00,2021-11-28 18:30:00,0,2021-11-28 18:30:00,0
2,2021-12-13 13:25:00,2021-12-13 15:40:00,0,2021-12-13 13:35:00,0
3,2021-12-13 13:25:00,2021-12-13 15:40:00,0,2021-12-13 14:15:00,0
4,2021-12-13 13:25:00,2021-12-13 15:40:00,0,2021-12-13 13:50:00,0
...,...,...,...,...,...
83780,2022-03-21 09:25:00,2022-03-21 11:40:00,193,2022-03-21 11:30:00,193
83781,2022-01-01 11:15:00,2022-01-01 13:30:00,197,2022-01-01 11:20:00,197
83782,2022-01-01 10:50:00,2022-01-01 13:05:00,197,2022-01-01 10:55:00,197
83783,2022-01-01 10:50:00,2022-01-01 13:05:00,197,2022-01-01 11:20:00,197


In [6]:
(out
 .sort_values(out.columns.tolist(), ignore_index=True)
 .equals(outer.sort_values(outer.columns.tolist(), ignore_index=True))
)

True

In [7]:
#https://stackoverflow.com/q/61948103/7175713 
df1 = pd.DataFrame({'id': [1,1,1,2,2,3], 
                    'value_1': [2,5,7,1,3,4]})

df2 = pd.DataFrame({'id': [1,1,1,1,2,2,2,3], 
                    'value_2A': [0,3,7,12,0,2,3,1], 
                    'value_2B': [1,5,9,15,1,4,6,3]})

df2 = df2.sample(len(df2))

# df1 = pd.concat([df1]*20)
# df2 = pd.concat([df2]*20)

In [8]:
df1.conditional_join(df2, ('id','id','=='), ('value_1','value_2A', '>'),('value_1', 'value_2B', '<'), use_numba=True)

left         right                  
    id value_1    id value_2A value_2B
0    2       3     2        2        4

In [9]:
df1.conditional_join(df2, ('id','id','=='), ('value_1', 'value_2A', '>'),('value_1', 'value_2B', '<'), use_numba=False)

left         right                  
    id value_1    id value_2A value_2B
0    2       3     2        2        4

In [10]:
df1

,id,value_1
0,1,2
1,1,5
2,1,7
3,2,1
4,2,3
5,3,4


In [11]:
df2

,id,value_2A,value_2B
4,2,0,1
3,1,12,15
1,1,3,5
5,2,2,4
2,1,7,9
7,3,1,3
0,1,0,1
6,2,3,6


Join on equi and non-equi operators is possible:

In [12]:
df1.conditional_join(
        df2,
        ('id', 'id', '=='),
        ('value_1', 'value_2A', '>='),
        ('value_1', 'value_2B', '<='),
        sort_by_appearance = True
    )

left         right                  
    id value_1    id value_2A value_2B
0    1       5     1        3        5
1    1       7     1        7        9
2    2       1     2        0        1
3    2       3     2        2        4
4    2       3     2        3        6

The default join is inner. left and right joins are supported as well:

In [13]:
df1.conditional_join(
        df2,
        ('id', 'id', '=='),
        ('value_1', 'value_2A', '>='),
        ('value_1', 'value_2B', '<='),
        how='left',
        sort_by_appearance = True
    )

left         right                  
    id value_1    id value_2A value_2B
0    1       2   NaN      NaN      NaN
1    1       5   1.0      3.0      5.0
2    1       7   1.0      7.0      9.0
3    2       1   2.0      0.0      1.0
4    2       3   2.0      2.0      4.0
5    2       3   2.0      3.0      6.0
6    3       4   NaN      NaN      NaN

In [14]:
df1.conditional_join(
        df2,
        ('id', 'id', '=='),
        ('value_1', 'value_2A', '>='),
        ('value_1', 'value_2B', '<='),
        how='right',
        sort_by_appearance = True
    )

left         right                  
    id value_1    id value_2A value_2B
0  2.0     1.0     2        0        1
1  NaN     NaN     1       12       15
2  1.0     5.0     1        3        5
3  2.0     3.0     2        2        4
4  1.0     7.0     1        7        9
5  NaN     NaN     3        1        3
6  NaN     NaN     1        0        1
7  2.0     3.0     2        3        6

Join on just the non-equi joins is also possible:

In [15]:
df1.conditional_join(
        df2,
        ('value_1', 'value_2A', '>'),
        ('value_1', 'value_2B', '<'),
        how='inner',
        sort_by_appearance = True
    )

left         right                  
    id value_1    id value_2A value_2B
0    1       2     3        1        3
1    1       5     2        3        6
2    2       3     2        2        4
3    3       4     1        3        5
4    3       4     2        3        6

Join on not equal -> !=

In [16]:
df1.conditional_join(
        df2,
        ('id', 'id', "!=")
    )

left         right                  
     id value_1    id value_2A value_2B
0     1       2     2        0        1
1     1       2     2        2        4
2     1       2     2        3        6
3     1       2     3        1        3
4     1       5     2        0        1
5     1       5     2        2        4
6     1       5     2        3        6
7     1       5     3        1        3
8     1       7     2        0        1
9     1       7     2        2        4
10    1       7     2        3        6
11    1       7     3        1        3
12    2       1     3        1        3
13    2       3     3        1        3
14    2       1     1       12       15
15    2       1     1        3        5
16    2       1     1        7        9
17    2       1     1        0        1
18    2       3     1       12       15
19    2       3     1        3        5
20    2       3     1        7        9
21    2       3     1        0        1
22    3       4     1       12       15
23    3       4     1        3        5
24    3       4     1        7        9
25    3       4     1        0        1
26    3       4     2        0        1
27    3       4     2        2        4
28    3       4     2        3        6

If the columns from both dataframes have nothing in common, a single indexed column is returned:

In [17]:
(df1.select_columns('value_1')
    .conditional_join(
        df2.select_columns('val*'),
        ('value_1', 'value_2A', '>'),
        ('value_1', 'value_2B', '<'),
    )
)

,value_1,value_2A,value_2B
0,2,1,3
1,5,3,6
2,3,2,4
3,4,3,5
4,4,3,6


Selection of relevant columns within `conditional_join`: 

In [18]:
df1.conditional_join(
        df2,
        ('id', 'id', "<"),
        df_columns = 'id',
        right_columns = 'id'
    )

,left,right
,id,id
0,1,2
1,1,2
2,1,2
3,1,3
4,1,2
5,1,2
6,1,2
7,1,3
8,1,2


Column renaming is also possible:

In [19]:
df1.conditional_join(
        df2,
        ('id', 'id', "<"),
        df_columns = {'id':'df_id'},
        right_columns = {'id':'right_id'}
    )

,df_id,right_id
0,1,2
1,1,2
2,1,2
3,1,3
4,1,2
5,1,2
6,1,2
7,1,3
8,1,2
9,1,2
